In [2]:
import sqlite3
import pandas as pd
import numpy as np

In [16]:
df = pd.read_excel('ITUDATA1.xlsx')
df1 = df[df['seriesCode'].isin(['xHH4_IDI'])]
#ICTTelephoneLinesPer100 = 391_i91
#ICTBroadbandSubscribersPer100ITU = 19303_i992b
#ICTTelephoneCellSubscribersPer100 = 392_i911
#ICTBroadbandMobileSubsPer100 = 12066_i911mw
#ICTInternetHousehold% = 12047_xHH6_IDI
#ICTComputerHousehold% = 12046_xHH4_IDI
# first part of code before _ is seriesID, second part is seriesCode

df2 = df1.drop(columns=['seriesID', 'seriesCode', 'seriesName', 'seriesParent', 'seriesUnits', 'entityID', 'entityIso', 'dataNote', 'dataSource', 'seriesDescription'])
df2 = df1.pivot_table(index = ["entityName"],
                    columns='dataYear',
                    values='dataValue')

#country concordance 
country = pd.read_excel('ITU Country.xlsx')
dt = country.merge(df2, on='entityName', how='left')
dt = dt.drop(columns='entityName')
dt = dt.rename(columns={'IFs Country':'Country'})
dt = dt.rename(columns={'dataYear':'Year'})


#dt = dt.reset_index()
dt.columns = [int(col) if str(col).replace('.', '', 1).isdigit() else col for col in dt.columns]

year_columns = [col for col in dt.columns if isinstance(col, int)]


dt['Earliest'] = dt[year_columns].bfill(axis=1).iloc[:, 0]
dt['MostRecent'] = dt[year_columns].ffill(axis=1).iloc[:, -1]

#change decimal place according to the datadict; all are four
dt[year_columns + ['Earliest', 'MostRecent']] = dt[year_columns + ['Earliest', 'MostRecent']].round(4)
dt

,Country,FIPS_CODE,1990,1991,1992,1993,1994,1995,1996,1997,...,2016,2017,2018,2019,2020,2021,2022,2023,Earliest,MostRecent
0,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2500,11.2833
1,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.8004,21.5774,20.2390,NaN,NaN,NaN,NaN,NaN,1.4000,20.2390
2,Algeria,DZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.8878,NaN,42.1723,NaN,NaN,NaN,NaN,NaN,2.1270,42.1723
3,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.0000,24.0000,32.2001,NaN,NaN,NaN,NaN,NaN,1.4851,32.2001
4,Argentina,ARG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65.9593,64.3112,63.0000,60.9183,63.7900,64.1467,62.62,60.99,20.5000,60.9900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42.7400,45.6891,NaN,NaN,NaN,NaN,NaN,NaN,5.1000,45.6891
184,Viet Nam,VNM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.5280,25.2400,32.2036,25.8000,27.9600,27.1300,28.29,26.99,2.4000,26.9900
185,Yemen,YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0241,7.4684,NaN,NaN,NaN,NaN,NaN,NaN,0.1829,7.4684
186,Zambia,ZMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.1000,NaN,NaN,NaN,NaN,NaN,0.7500,8.1000


In [ ]:
#sql table; change table name each time you import 
conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport - Copy (2).db')
cursor = conn.cursor()
table_name = 'SeriesICTComputerHousehold%'

#ICTDevelopmentIndexITU = ICT Development Index (IDI)
#ICTTelephoneLinesPer100 = 391_i91
#ICTBroadbandSubscribersPer100ITU = 19303_i992b
#ICTTelephoneCellSubscribersPer100 = 392_i911
#ICTBroadbandMobileSubsPer100 = 12066_i911mw
#ICTInternetHousehold% = 12047_xHH6_IDI
#ICTComputerHousehold% = 12046_xHH4_IDI

columns_sql = ['Country VARCHAR(255)', 'FIPS_CODE VARCHAR(255)']
columns_sql += [f'"{col}" DOUBLE(53)' for col in year_columns + ['Earliest', 'MostRecent']]
create_table_sql = f"CREATE TABLE '{table_name}' ({', '.join(columns_sql)})"
cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
cursor.execute(create_table_sql)

dt.to_sql(table_name, conn, if_exists='append', index=False)

conn.commit()
conn.close()

In [42]:
idi = pd.read_excel('IDIDataset_2025.xlsx', sheet_name='Sheet1')

idi1 = idi.pivot_table(index = ["Economy"],
                    columns='Year',
                    values='Value')

#country concordance 
country = pd.read_excel('ITU Country.xlsx',sheet_name='IDI')
dt = country.merge(idi1, on='Economy', how='left')
dt = dt.drop(columns='Economy')

dt.columns = [int(col) if str(col).replace('.', '', 1).isdigit() else col for col in dt.columns]

year_columns = [col for col in dt.columns if isinstance(col, int)]


dt['Earliest'] = dt[year_columns].bfill(axis=1).iloc[:, 0]
dt['MostRecent'] = dt[year_columns].ffill(axis=1).iloc[:, -1]

#change decimal place according to the datadict; all are four
dt[year_columns + ['Earliest', 'MostRecent']] = dt[year_columns + ['Earliest', 'MostRecent']].round(4)
dt

,Country,FIPS_CODE,2023,2024,2025,Earliest,MostRecent
0,Afghanistan,AFG,28.9,33.1,36.5,28.9,36.5
1,Albania,ALB,81.6,84.7,84.3,81.6,84.3
2,Algeria,DZA,77.8,80.9,86.1,77.8,86.1
3,Angola,AGO,44.1,49.9,52.8,44.1,52.8
4,Argentina,ARG,81.5,83.4,83.7,81.5,83.7
5,Armenia,ARM,85.1,86.4,86.9,85.1,86.9
6,Australia,AUS,94.0,95.1,95.8,94.0,95.8
7,Austria,AUT,92.5,94.3,95.5,92.5,95.5
8,Azerbaijan,AZE,79.0,80.4,85.9,79.0,85.9
9,Bahamas,BHS,88.5,89.3,88.9,88.5,88.9


In [43]:
#sql table; change table name each time you import 
conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport - Copy (2).db')
cursor = conn.cursor()
table_name = 'SeriesICTDevelopmentIndexITU'

#ICTDevelopmentIndexITU = ICT Development Index (IDI)

columns_sql = ['Country VARCHAR(255)', 'FIPS_CODE VARCHAR(255)']
columns_sql += [f'"{col}" DOUBLE(53)' for col in year_columns + ['Earliest', 'MostRecent']]
create_table_sql = f"CREATE TABLE '{table_name}' ({', '.join(columns_sql)})"
cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
cursor.execute(create_table_sql)

dt.to_sql(table_name, conn, if_exists='append', index=False)

conn.commit()
conn.close()

In [ ]:
# importig datadict
DataDict = pd.read_csv('datadict.csv')

conn = sqlite3.connect('IFsDataImport.db')
cursor = conn.cursor()
table_name = 'DataDict'

DataDict.to_sql(table_name, conn, if_exists='append', index=False)
conn.commit()
conn.close()